In [7]:
import pandas as pd
import numpy as np

## Recomendação Simples - Geral - Filtro de Popularidade(Sistema de Recomendação)


Esse algoritmo simples oferece recomendações generalizadas para cada curso baseado em sua popularidade.

A ideia básica por trás desse recomendador é que os cursos mais populares e aclamados terão maiores probabilidades de serem apreciados pela média do público que possuem as habilidades que os parceiros da Bettha consideram mais importantes para o mercado.

**Nota**: Esse modelo não dá recomendações baseadas no usuário.

### Puxando Dados

In [8]:
from google.colab import drive
drive.mount('/content/drive')

tabelas = '/content/drive/Shareddrives/BTECH/BTECH/Tabelas'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
df_journeys = pd.read_csv(tabelas + '/journeys_inteli.csv')
df_journeys.head()

,id,name,type,category_type,average_rating,ratings_count,competencias
0,309,Visão de Negócios | Jornada para o Futuro,increase_employability,content,5.0,65,improve_dis - improve_sin
1,103,Comunicação escrita - será que você realmente ...,portable_skills,content,4.5,264,improve_eng - improve_sin - improve_int
2,38,Como você faz suas escolhas de carreira? #anco...,know_myself_better,content,4.5,313,improve_int - improve_res
3,108,Gestão de Projetos - um passo de cada vez,portable_skills,content,4.5,274,improve_int - improve_neg
4,708,Saiba mais sobre a Brainfarma,increase_employability,content,5.0,102,emp


### Pré-Processamento de Dados

In [10]:
df_journeys.shape

(209, 7)

In [11]:
df_journeys.isna().sum()

id                0
name              0
type              0
category_type     0
average_rating    0
ratings_count     0
competencias      2
dtype: int64

Sabendo que o número de linhas com valores nulos na coluna 'competências' é extremamente baixo, com base no total de dados em nosso dataframe, podemos descartá-los

In [12]:
df_journeys.dropna(subset=['competencias'], inplace=True)
df_journeys.isna().sum()

id                0
name              0
type              0
category_type     0
average_rating    0
ratings_count     0
competencias      0
dtype: int64

### Mão na Massa:

#### Fórmula para classificar dados

Vamos usar a fórmula de classificação ponderada do IMDB para o nosso recomendador.

Matematicamente, ela é representada da seguinte forma:

$$ Avaliação Ponderada (AP) =  (vv+m.R)+(mv+m.C) $$

onde:
- v é o número de votos para o curso
- m é o número mínimo de votos necessários para ser listado
- R é a classificação média do curso
- C é a média dos votos em todo o relatório




In [13]:
rating_counts = df_journeys[df_journeys['ratings_count'].notnull()]['ratings_count'].astype(int)
rating_average = df_journeys[df_journeys['average_rating'].notnull()]['average_rating'].astype(int)

#### Determinando o valor de `m`:

Para determinar um valor adequado para m, ou seja, o número mínimo de votos necessários para ser listado no dataframe, usaremos o 95º percentil como nosso ponto de corte.

Em outras palavras, para que um curso seja incluído no dataframe, ele deve ter mais votos do que pelo menos 95% dos cursos na lista.







In [14]:
m = rating_counts.quantile(0.95)
m

1921.8999999999965

**O que este número representa???** 🤔

Isso significa que, para ser considerado pelo classificador, um curso deve ter pelo menos 1921 votos!!!


Primeiro, precisamos verificar quantas avaliações um curso na plataforma da Bettha possui. Como podemos ver, o curso com mais avaliações tem mais de 90 mil e o menor tem 0. Portanto, para que um curso se qualifique como um dos 5% melhores, ele deve ter pelo menos 1921 avaliações.





In [15]:
df_journeys.sort_values(['ratings_count'], ascending=False)

,id,name,type,category_type,average_rating,ratings_count,competencias
14,86,Sua Primeira Jornada,know_myself_better,assessment,5.0,93008,plataforma
164,25,Teste o seu Inglês,increase_employability,tests,4.5,24880,plataforma
204,88,Chave Mestra,know_myself_better,assessment,5.0,23278,plataforma
8,158,Complemento de Cadastro,know_myself_better,register,4.5,19132,plataforma
29,385,Soft skills: protagonismo e resiliência,portable_skills,content,5.0,5610,improve_res - improve_cur - improve_int
...,...,...,...,...,...,...,...
152,729,"Afinal, por que falar de tempo? - Goodman",portable_skills,content,5.0,1,improve_sin - improve_dis
156,776,2: Como eu me comunico? | Comunicação Goodman,soft_skills,content,5.0,1,improve_int
190,870,Protagonismo e Resiliência com o Bettha | Dn’A...,portable_skills,content,5.0,1,improve_res - improve_eng - improve_cur
66,775,1: O que é? | Comunicação Goodman,soft_skills,content,4.0,1,improve_int - improve_eng


#### Determinando o valor de `C`:

In [16]:
C = rating_average.mean()
C

4.714975845410628

Podemos também perceber que a avaliação média é de 4.714.

#### Filtrando dados com base no `m`:

In [17]:
filter = df_journeys[(df_journeys['ratings_count'] >= m) & (df_journeys['ratings_count'].notnull()) & (df_journeys['average_rating'].notnull())][['name', 'type', 'ratings_count', 'average_rating', 'competencias']]
filter['ratings_count'] = filter['ratings_count'].astype('int')
filter['average_rating'] = filter['average_rating'].astype('int')
filter.shape

(11, 5)

Notas:
- Apenas 11 cursos podem ser classificados para fazer parte do nosso novo dataframe.
- Mais uma vez, isso é apenas um exemplo. Se mudarmos o percentil, a quantidade de cursos recomendados também mudará.

In [18]:
def weighted_rating(x):
    v = x['ratings_count']
    R = x['average_rating']
    return (v/(v+m) * R) + (m/(m+v) * C)

In [19]:
filter['wr'] = filter.apply(weighted_rating, axis=1)



#### Utilizamos a variável `show` para mostrar os 10 cursos mais recomendados, dando à variável valor 10.

In [20]:
show = 10

In [21]:
qualified = filter.sort_values('wr', ascending=False).head(show)

In [22]:
qualified.head(show)

,name,type,ratings_count,average_rating,competencias,wr
14,Sua Primeira Jornada,know_myself_better,93008,5,plataforma,4.994230
204,Chave Mestra,know_myself_better,23278,5,plataforma,4.978262
29,Soft skills: protagonismo e resiliência,portable_skills,5610,5,improve_res - improve_cur - improve_int,4.927271
93,SOLU - Teste de lógica,pss_engineering,4589,5,plataforma,4.915866
96,Conheça mais sobre o Potência Tech,increase_employability,4392,5,emp,4.913241
194,Sobre a empresa: Programa do Estágio Nestlé 2023,increase_employability,3975,5,emp,4.907106
186,Saiba mais sobre a Ocyan,increase_employability,2564,5,emp,4.877887
195,Saiba mais sobre a Mondelez International,increase_employability,2251,5,emp,4.868727
160,Soft skills | Ocyan,portable_skills,2014,5,improve_int - emp,4.860823
8,Complemento de Cadastro,know_myself_better,19132,4,plataforma,4.065266




## Recomendador Simples - Coluna Específica - Filtro de Popularidade (Sistema de Recomendação)
Desta vez, vamos construir nosso recomendador que cria gráficos para uma coluna específica.

Para isso, vamos relaxar nossas condições padrão e usar o 85º percentil em vez do 95º.

**Nota**: Este modelo não fornece recomendações personalizadas com base no usuário.

O código abaixo pode ser usado ou testado com qualquer outra coluna, mas para dar um exemplo e aprimorar nosso entendimento sobre ele, vamos responder à próxima pergunta:

**Quais são os cursos mais bem classificados do tipo: know_myself_better?**

In [23]:
column_name = 'type'
column_name_df_journeys = df_journeys.explode(column_name)

In [24]:
def build_chart(value, column_name, percentile=0.85):
    df = column_name_df_journeys[column_name_df_journeys[column_name] == value]

    qualified = df[(df['ratings_count'].notnull()) & (df['average_rating'].notnull())]
    qualified['ratings_count'] = qualified['ratings_count'].astype(int)
    qualified['average_rating'] = qualified['average_rating'].astype(int)

    C = qualified['average_rating'].mean()
    m = qualified['ratings_count'].quantile(percentile)

    qualified['wr'] = (qualified['ratings_count'] / (qualified['ratings_count'] + m) * qualified['average_rating']) + (m / (m + qualified['ratings_count']) * C)

    qualified = qualified.sort_values('wr', ascending=False)

    return qualified

### Let's test:

Now, we just have to test the code above and see the result.


`build_chart arguments:`
- value: this variable represents what information we are looking for in determined column
- column_name: in what column I can search some information


### Vamos testar o código acima e ver o resultado.

`Argumentos para a função build_chart:`

value: esta variável representa que tipo de informação estamos procurando na coluna determinada.
column_name: em qual coluna posso procurar por essa informação.

In [25]:
value = 'know_myself_better'

In [26]:
build_chart(value, column_name).head()

,id,name,type,category_type,average_rating,ratings_count,competencias,wr
14,86,Sua Primeira Jornada,know_myself_better,assessment,5,93008,plataforma,4.996194
204,88,Chave Mestra,know_myself_better,assessment,5,23278,plataforma,4.985130
17,241,Gestão adaptativa,know_myself_better,content,5,670,improve_int - improve_res,4.742140
88,293,O que é o Genius?,know_myself_better,content,5,296,improve_dis - plataforma,4.646607
121,294,O que é o Lifestyle?,know_myself_better,content,5,277,improve_eng,4.639828
